# Targets for Abi

She's on at Magellan June 3 and 22, 2022, and needs second-half-night targets.

| field | ra | dec|
|------------|--------|----------|
|COSMOS-1    | 150.0  |  3.1     |
|COSMOS-2    | 149.22 | 1.75     |
|COSMOS-3    | 150.78 | 1.75     |
|DECaPS-East | 270.89 | -29.53   |
|ELAIS-E1    | 7.8746 | -43.0096 |
|ELAIS-E2    | 9.5    | -43.9980 |
|DECaPS-West | 116.32 | -26.25   |

# try for ELAIS

In [ ]:
import psycopg2
import psycopg2.extras
import getpass
# from IPython.display import display
# import ipywidgets

import os
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
# import pandas as pd

from copy import deepcopy
import time
from astropy.time import Time

font = {'size' : 15}
mpl.rc('font', **font)

In [ ]:
dbuser = input("DB User: ")
dbpasswd = getpass.getpass("DB Password: ")

In [ ]:
db = psycopg2.connect(f"dbname='decat' user='{dbuser}' password='{dbpasswd}' host='decatdb.lbl.gov'")

db.autocommit = True # Makes the computer nicer about SQL syntax errors

cursor = db.cursor( cursor_factory = psycopg2.extras.DictCursor )

In [ ]:
### If you want to print table schema
# tables = ['exposures','subtractions','objects','objectrbs','candidates']
tables = ['exposures','subtractions','objects','candidates']
for table in tables:
    query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name=%s"
    cursor.execute( query, ( table, ))
    print( f"\nTABLE: {table}\n===========================" )
    for row in cursor:
        print( f"{row['column_name']:24s}  :  {row['data_type']:s}" )

### Look for recent candidates in ELAIS

|ELAIS-E1    | 7.8746 | -43.0096 |

|ELAIS-E2    | 9.5    | -43.9980 |


In [ ]:
t1 = time.time()

query = ("SELECT id, ra, dec, created_at, modified FROM candidates "
         "WHERE ra > 6 AND ra < 11 AND dec < -41 AND dec > -45")

cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

cand_id       = np.asarray( results[0] )
cand_ra       = np.asarray( results[1], dtype='float' )
cand_dec      = np.asarray( results[2], dtype='float' )
cand_created  = np.asarray( results[3], dtype='str' )
cand_modified = np.asarray( results[4], dtype='str' )
del query, results

print(len(cand_id))

In [ ]:
print('Dates of ELAIS candidates, first and last created.')
sx = np.argsort( cand_created )
print(cand_created[sx[0]])
print(cand_created[sx[-1]])
print(' ')
print('Dates of ELAIS candidates, first and last modified.')
sx = np.argsort( cand_modified )
print(cand_modified[sx[0]])
print(cand_modified[sx[-1]])
print(' ')

### Look for recent objects in ELAIS

In [ ]:
t1 = time.time()

query = ("SELECT o.id, o.ra, o.dec, o.candidate_id, o.subtraction_id, o.mag, o.magerr, "
         "rbs.rb, sub.exposure_id, exp.mjd, exp.filter "
         "FROM objects AS o "
         "JOIN objectrbs AS rbs ON o.id=rbs.object_id AND rbs.rbtype_id=1 "
         "JOIN subtractions AS sub ON sub.id=o.subtraction_id "
         "JOIN exposures AS exp ON exp.id=sub.exposure_id "
         "WHERE o.ra > 6 AND o.ra < 11 AND o.dec < -41 AND o.dec > -45 "
         "AND exp.mjd > 59680")
cursor.execute( query )
results = np.array( cursor.fetchall() ).transpose()

t2 = time.time()
print(t2-t1)
del t1,t2

obj_id     = np.asarray( results[0], dtype='int' )
obj_ra     = np.asarray( results[1], dtype='float' )
obj_dec    = np.asarray( results[2], dtype='float' )
obj_candid = np.asarray( results[3] )
obj_subid  = np.asarray( results[4], dtype='int' )
obj_mag    = np.asarray( results[5], dtype='float' )
obj_magerr = np.asarray( results[6], dtype='float' )
obj_rb     = np.asarray( results[7], dtype='float' )
obj_expid  = np.asarray( results[8], dtype='int' )
obj_mjd    = np.asarray( results[9], dtype='float' )
obj_filter = np.asarray( results[10] )

del query, results

print(len(obj_id))